In [85]:
using Plots, Flux, DataFrames, CSV, ProgressMeter, Statistics
import MLUtils: splitobs, normalise
import BSON: @save
max_epochs = 10
frac_train = 0.6

0.6

In [86]:
df  = CSV.read("data/Landslides.csv", DataFrame)
display(df)

Row,dist_roads,DEM,TWI,plan_curvature,profil_curvature,Slope,Geology,LandCover,LS
,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Int64,Int64
1,90.1388,957.1,10.0174,0.00257542,1.56e-5,11.0455,2,41,0
2,282.843,536.1,9.21162,-0.00012923,-0.000289093,3.57188,1,21,0
3,90.1388,768.1,10.0824,0.00623542,-0.00240442,44.0343,2,41,0
4,79.0569,449.4,7.35267,0.00420136,0.00116142,11.3254,24,21,1
5,25.0,784.8,8.60168,0.00208871,-0.00271119,19.0155,1,41,1
6,25.0,991.5,13.7942,-0.00129205,0.00126791,8.28275,9,21,0
7,35.3553,569.2,8.86016,-0.00346738,0.00069258,23.9199,24,31,1
8,70.7107,1211.4,9.46106,-0.00180184,-0.00452195,21.2627,9,21,0
9,413.824,1592.5,8.28995,-0.00207019,0.00560989,38.1138,7,31,1


In [87]:
# One hot batches for categorical variables
LandCover = Flux.onehotbatch(Array(df[:,8]), unique(Array(df[:,8])))
Geology   = Flux.onehotbatch(Array(df[:,7]), unique(Array(df[:,7])))

9×5185 OneHotMatrix(::Vector{UInt32}) with eltype Bool:
 1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  1  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  1  ⋅  ⋅     ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  1  ⋅  1
 ⋅  ⋅  ⋅  1  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  1  1     ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  1  1  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅  ⋅  ⋅  ⋅     ⋅  1  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  …  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  1  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅
 ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅     ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅  ⋅

In [88]:
# Vectors of length 2, as columns of a matrix:
# noisy     = [Array(df[1:500,1:6])'; Geology_train; LandCover_train] # 2×1000 Matrix{Float32}
# truth     = Array(df[1:500,9]).==1;  #   1000-element Vector{Bool}
X_all  = [ Array(df[:,1:6])'; Geology; LandCover ]
X_all  = Array(df[:,1:6])'#; Geology; LandCover ]
y_all  = df[:,9].==1
# Flux.normalise(X_all)
# noisy  = X_all[:,1:500]
# truth  = y_all[1:500]
# target = Flux.onehotbatch(truth, [true, false]) 
# println( (size(noisy), size(truth)) )

# noisy_test  = X_all[:,501:end]
# truth_test  = y_all[501:end]

# Split with multiple arrays and shuffling
train, test = splitobs((X_all, y_all), at=frac_train, shuffle=true)
X_train, Y_train = train
X_test,  Y_test  = test
noisy       = X_train
target      = Flux.onehotbatch(Y_train, [true, false]) 
noisy_test  = X_test
target_test = Flux.onehotbatch(Y_test, [true, false]) 

X_all

6×5185 adjoint(::Matrix{Float64}) with eltype Float64:
  90.1388      282.843         90.1388      …  458.939       500.0
 957.1         536.1          768.1            895.5         576.2
  10.0174        9.21162       10.0824           8.91429       9.6554
   0.00257542   -0.00012923     0.00623542      -0.003389      0.000434234
   1.56e-5      -0.000289093   -0.00240442       0.00749098    0.000594097
  11.0455        3.57188       44.0343      …   33.9126        2.29349

In [89]:
# model = Chain(
#     Dense(22 => 25, tanh),   # activation function inside layer
#     BatchNorm(25),
#     Dense(25 => 22),
#     BatchNorm(22),
#     Dense(22 => 2),
#     softmax) 

model = Chain(
    Dense(6 => 25, tanh),   # activation function inside layer     
    BatchNorm(25),
    Dense(25 => 2),
    softmax
    ) 

Chain(
  Dense(6 => 25, tanh),                 # 175 parameters
  BatchNorm(25),                        # 50 parameters, plus 50
  Dense(25 => 2),                       # 52 parameters
  NNlib.softmax,
)         # Total: 6 trainable arrays, 277 parameters,
          # plus 2 non-trainable, 50 parameters, summarysize 1.754 KiB.

In [90]:
out1 = model( noisy );

In [91]:
pars = Flux.params(model)  # contains references to arrays in model
opt  = Flux.Adam(0.05)      # will store optimiser momentum, etc.

Adam(0.05, (0.9, 0.999), 1.0e-8, IdDict{Any, Any}())

In [92]:
# To train the model, we use batches of 64 samples, and one-hot encoding:
# target = Flux.onehotbatch(truth, [true, false])                   # 2×1000 OneHotMatrix
loader = Flux.DataLoader((noisy, target), batchsize=64, shuffle=true);
# 16-element DataLoader with first element: (2×64 Matrix{Float32}, 2×64 OneHotMatrix)

49-element DataLoader(::Tuple{SubArray{Float64, 2, LinearAlgebra.Adjoint{Float64, Matrix{Float64}}, Tuple{Base.Slice{Base.OneTo{Int64}}, Vector{Int64}}, false}, OneHotArrays.OneHotMatrix{UInt32, Vector{UInt32}}}, shuffle=true, batchsize=64)
  with first element:
  (6×64 Matrix{Float64}, 2×64 OneHotMatrix(::Vector{UInt32}) with eltype Bool,)

In [93]:
loss1(model, x, y) = Flux.crossentropy(model(x), y);
loss1(model, noisy, target)
accuracy(x, y) = mean(Flux.onecold(model(x)) .== Flux.onecold(y)) # Define the accuracy 

accuracy (generic function with 1 method)

In [94]:
@show loss0     = loss1(model, noisy, target)
@show acc0      = accuracy(noisy, target)
@show acc_test0 = accuracy(noisy_test, target_test)

loss0 = loss1(model, noisy, target) = 1.587354654538221
acc0 = accuracy(noisy, target) = 0.49823207971713274


acc_test0 = accuracy(noisy_test, target_test) = 0.4811957569913211


0.4811957569913211

In [103]:
# Training loop, using the whole data set 1000 times:
losses          = []
accuracies      = []
losses_test     = []
accuracies_test = []
for epoch in 1:max_epochs #@showprogress 
    for (x, y) in loader
        loss, grad = Flux.withgradient(pars) do
            # Evaluate model and loss inside gradient context:
            y_hat = model(x)
            Flux.crossentropy(y_hat, y)
        end
        Flux.update!(opt, pars, grad)
        #####
        push!(losses, loss)  # logging, outside gradient context
        accu=accuracy(noisy, target)
        push!(accuracies, accu)
        #####
        loss_test = loss1(model, noisy_test, target_test)
        push!(losses_test, loss_test)  # logging, outside gradient context
        accu_test =accuracy(noisy_test, target_test)
        push!(accuracies_test, accu_test)
    end
end
@show loss1(model, noisy, target)
@show accuracy(noisy, target)
@show accuracy(noisy_test, target_test)
# println(noisy_test[:,1])

loss1(model, noisy, target) = 0.6939679965220837
accuracy(noisy, target) = 0.4927675988428158
accuracy(noisy_test, target_test) = 0.5115718418514947


0.5115718418514947

In [96]:
out2 = model(noisy)  # first row is prob. of true, second row p(false)
@show mean((out2[1,:] .> 0.5) .== Y_train)  # accuracy 94% so far!
@show mean((out2[1,:] .< 0.5) .== Y_train)  # accuracy 94% so far!
@show mean((out2[2,:] .> 0.5) .!= Y_train)  # accuracy 94% so far!
@show mean((out2[2,:] .< 0.5) .!= Y_train)  # accuracy 94% so far!
# mean((out2[1,:] .< 0.5) .!= Y_train)  # accuracy 94% so far!
# mean((out2[1,:] .> 0.5) .!= Y_train)  # accuracy 94% so far!

mean((out2[1, :] .> 0.5) .== Y_train) = 0.5072324011571842
mean((out2[1, :] .< 0.5) .== Y_train) = 0.4927675988428158
mean((out2[2, :] .> 0.5) .!= Y_train) = 0.5072324011571842
mean((out2[2, :] .< 0.5) .!= Y_train) = 0.4927675988428158


0.4927675988428158

In [97]:
# p_true = scatter(noisy[1,:], noisy[2,:], zcolor=Y_train,   title="True classification", legend=false, c=:turbo, xlabel=names(df)[2], ylabel=names(df)[3] )
# p_raw =  scatter(noisy[1,:], noisy[2,:], zcolor=out1[1,:], title="Untrained network", label="", clims=(0,1), c=:turbo, xlabel=names(df)[2], ylabel=names(df)[3] )
# p_done = scatter(noisy[1,:], noisy[2,:], zcolor=out2[1,:], title="Trained network", legend=false, c=:turbo, xlabel=names(df)[2], ylabel=names(df)[3] )

# plot(p_true, p_raw, p_done, layout=(1,3), size=(1000,400))

In [98]:
# plot(losses; xaxis=(:log10, "iteration"),
#     yaxis="loss", label="per batch")
# @show n = length(loader)
# plot!(n:n:length(losses), mean.(Iterators.partition(losses, n)),
#     label="epoch mean", title="Loss", dpi=200)

In [99]:
# plot(accuracies; xaxis=(:log10, "iteration"),
#     yaxis="loss", label="per batch")
# @show n = length(loader)
# plot!(n:n:length(accuracies), mean.(Iterators.partition(accuracies, n)),
#     label="epoch mean", title="Accuracy", dpi=200)